In [1]:
# Import libraries
import os
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Model pipeline
def opt_model_pipe(data=None, ini_date=None, end_date=None, ticker=['AAPL','MSFT','AMZN','TSLA','GOOGL','GOOG','NVDA','BRK-B','META','UNH'], initial_guess=None, output_part=None):
    # Get stats
    def stats(weights):
        weights = np.array(weights)
        expected_return = np.sum((log_returns.mean()*weights) * 252)
        expected_vol = np.sqrt(np.dot(weights.T,np.dot(log_returns.cov()*252,weights)))
        sharpe_r = expected_return/expected_vol
        return np.array([expected_return,expected_vol,sharpe_r])

    # Minimize negative Sharpe Ratio
    def sr_negate(weights):
        neg_sr = stats(weights)[2] * -1
        return neg_sr

    # Check allocation sums to 1
    def weight_check(weights):
        weights_sum = np.sum(weights)
        return weights_sum - 1

    # Generate input data from data lake
    def get_data(data, ticker=['AAPL','MSFT','AMZN','TSLA','GOOGL','GOOG','NVDA','BRK-B','META','UNH']):
        input_data = pd.DataFrame()
        for i in ticker:
            ticker_adj_close = data[data['Ticker']==i]['Adj Close']
            input_data = pd.concat((input_data,ticker_adj_close),axis=1)
        input_data.columns = ticker
        input_data.sort_index(inplace=True)
        return input_data

    # Generate model params
    def get_params(ticker=['AAPL','MSFT','AMZN','TSLA','GOOGL','GOOG','NVDA','BRK-B','META','UNH']):
        bounds = []
        initial_guess = []
        w = 1 / len(ticker)
        for i in ticker:
            bounds.append((0,1))
            initial_guess.append(w)
        return tuple(bounds), initial_guess
    
    # Model input data
    if data is None:
        # From Data Lake
        gold = './data/gold/portfolio-optimization/'
        gold_table = 'portfolio_optimization.csv'

        data = pd.read_csv(gold+gold_table)
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)
        
        if ((ini_date is not None) & (end_date is not None)):
            data[((data.index>=datetime.strptime(ini_date, '%Y-%m-%d')) & (data.index<=datetime.strptime(end_date, '%Y-%m-%d')))]
            data_range = '{} - {}'.format(ini_date, end_date)
        else:
            ini_date = data.index[0]
            end_date = data.index[-1]
            data_range = '{} - {}'.format(ini_date, end_date)
        
        input_data = get_data(data, ticker)
        input_data_msg = 'Data Lake'
    else:
        # From parameter data
        input_data = get_data(data, ticker)
        input_data_msg = 'User data input'
        data_range = 'N/A'
    
    # Model params
    if initial_guess is None:
        bounds, initial_guess = get_params(ticker)
    else:
        bounds, dummy = get_params(ticker)
    constraints = ({'type':'eq','fun':weight_check})
    
    # Logarithmic return
    log_returns = np.log(input_data/input_data.shift(1))
    
    # Model execution
    results = minimize(sr_negate,initial_guess,method='SLSQP',bounds=bounds,constraints=constraints)
    
    # Portfolio allocation weights and stats
    weights = list(results.x)
    stats = list(stats(results.x))
    
    # Generate JSON output
    if output_part == 0:
        output = {
            'ticker': ticker,
            'weights': weights
        }
    elif output_part == 1:
        output = {
            'return': stats[0],
            'volatility': stats[1],
            'sharpe_ratio': stats[2]
        }
    else:
        output = {
            'ticker': ticker,
            'weights': weights,
            'return': stats[0],
            'volatility': stats[1],
            'sharpe_ratio': stats[2],
            'input_data': input_data_msg,
            'data_range': data_range,
            'message': results.message,
            'success': str(results.success)
        }
    output_json = json.dumps(output)
    
    return output

In [3]:
# Data Lake params
silver = './data/silver/stock-prices/'
gold = './data/gold/portfolio-optimization/'
silver_table = 'stock_prices.csv'
gold_table = 'portfolio_optimization.csv'

In [4]:
# Read data from data lake
data = pd.read_csv(silver+silver_table)
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data

,Open,High,Low,Close,Adj Close,Volume,Ticker
Date,,,,,,,
2003-05-21,0.317679,0.323036,0.315536,0.318750,0.270919,305009600,AAPL
2003-05-22,0.319464,0.328571,0.316786,0.325714,0.276838,178460800,AAPL
2003-05-23,0.325179,0.329643,0.320714,0.327143,0.278053,206718400,AAPL
2003-05-27,0.320714,0.337500,0.319821,0.337143,0.286552,290130400,AAPL
2003-05-28,0.330357,0.333214,0.324107,0.326429,0.277446,339679200,AAPL
...,...,...,...,...,...,...,...
2023-05-15,4126.649902,4141.250000,4110.270020,4136.279785,4136.279785,3562170000,^GSPC
2023-05-16,4127.950195,4135.540039,4109.859863,4109.899902,4109.899902,3654200000,^GSPC
2023-05-17,4122.850098,4164.669922,4113.620117,4158.770020,4158.770020,4039080000,^GSPC


In [5]:
# Load data to data lake (gold layer)
gld_data = data[['Adj Close', 'Ticker']]
gld_data.to_csv(gold+gold_table)
gld_data

,Adj Close,Ticker
Date,,
2003-05-21,0.270919,AAPL
2003-05-22,0.276838,AAPL
2003-05-23,0.278053,AAPL
2003-05-27,0.286552,AAPL
2003-05-28,0.277446,AAPL
...,...,...
2023-05-15,4136.279785,^GSPC
2023-05-16,4109.899902,^GSPC
2023-05-17,4158.770020,^GSPC


In [6]:
# Ticker data available
gld_data.groupby('Ticker').size()

Ticker
AAPL     5035
AMZN     5035
BRK-B    5035
GOOG     4721
GOOGL    4721
META     2769
MSFT     5035
NVDA     5035
TSLA     3246
UNH      5035
^GSPC    5035
dtype: int64

In [7]:
# Historical data available
gld_data['Year']=gld_data.index.astype(str).str.slice(0, 7)
gld_data.groupby('Year').size()

Year
2003-05     49
2003-06    147
2003-07    154
2003-08    147
2003-09    147
          ... 
2023-01    220
2023-02    209
2023-03    253
2023-04    209
2023-05    165
Length: 241, dtype: int64

In [8]:
# Model params
ticker = ['AAPL','MSFT','AMZN','TSLA','GOOGL','GOOG','NVDA','BRK-B','META','UNH']
initial_guess = [0.15, 0.15, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05]
ini_date = '2020-01-01'
end_date = '2021-01-01'

# Model pipe execution
output = opt_model_pipe()
print(json.dumps(output, indent=2))

{
  "ticker": [
    "AAPL",
    "MSFT",
    "AMZN",
    "TSLA",
    "GOOGL",
    "GOOG",
    "NVDA",
    "BRK-B",
    "META",
    "UNH"
  ],
  "weights": [
    0.5864107248923445,
    2.6725583551767684e-17,
    0.003608026578021744,
    0.16129143539688615,
    0.0,
    0.1127960642632393,
    5.800481622797449e-18,
    5.976664475826343e-18,
    4.8856860397628044e-17,
    0.1358937488695083
  ],
  "return": 0.2950527693040578,
  "volatility": 0.2799861424312236,
  "sharpe_ratio": 1.0538120449176702,
  "input_data": "Data Lake",
  "data_range": "2003-05-21 00:00:00 - 2023-05-19 00:00:00",
  "message": "Optimization terminated successfully",
  "success": "True"
}


In [9]:
# Model params
ticker = ['AAPL','MSFT','AMZN','TSLA','GOOGL','GOOG','NVDA','BRK-B','META','UNH']
initial_guess = [0.15, 0.15, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05]
ini_date = '2020-01-01'
end_date = '2021-01-01'

# Model pipe execution
output = opt_model_pipe(data=gld_data, ini_date=ini_date, end_date=end_date, ticker=ticker, initial_guess=initial_guess, output_part=0)
print(json.dumps(output, indent=2))

{
  "ticker": [
    "AAPL",
    "MSFT",
    "AMZN",
    "TSLA",
    "GOOGL",
    "GOOG",
    "NVDA",
    "BRK-B",
    "META",
    "UNH"
  ],
  "weights": [
    0.5864156597653565,
    0.0,
    0.003611996028247581,
    0.1613065173983884,
    2.196593601455632e-16,
    0.11278317776055792,
    0.0,
    0.0,
    3.767602549387128e-17,
    0.1358826490474494
  ]
}


In [10]:
# Data tabulation
pd.read_json(json.dumps(output))

,ticker,weights
0,AAPL,5.864157e-01
1,MSFT,0.000000e+00
2,AMZN,3.611996e-03
3,TSLA,1.613065e-01
4,GOOGL,2.196594e-16
5,GOOG,1.127832e-01
6,NVDA,0.000000e+00
7,BRK-B,0.000000e+00
8,META,3.767603e-17
9,UNH,1.358826e-01
